[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shashist/Computational-math/blob/master/Interpolation.ipynb)

In [1]:
!pip install -q rs_datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 74.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 79.9 MB/s eta 0:00:00


In [34]:
from collections import defaultdict
from copy import deepcopy

import numpy as np
import pandas as pd
import scipy.sparse as sp
from rs_datasets import MovieLens
from scipy.sparse import csr_matrix, dok_matrix

## 0. MovieLens-1M dataset

- probably the most popular dataset in recommender systems
- `user_id` ranges from 1 to 6040
- `item_id` ranges from 1 to 3952
- 1000209 ratings available

In [3]:
%%time
movielens = MovieLens('1m')
movielens.info()

INFO:rs_datasets:Downloading ml-1m from grouplens...
5.93MB [00:00, 20.8MB/s]                           


ratings


,user_id,item_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968



users


,user_id,gender,age,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117



items


,item_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance



CPU times: user 1.43 s, sys: 424 ms, total: 1.86 s
Wall time: 2.58 s


Dataset has different versions, more details in [paper](http://files.grouplens.org/papers/harper-tiis2015.pdf)

<img src="https://raw.githubusercontent.com/shashist/recsys-course/master/week_02_neighbourhood_based/ml_versions.png" width=700>

## 1. Validation strategy (date split)

In [4]:
log = movielens.ratings
log.head(2)

,user_id,item_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109


In [5]:
time_treshold = log['timestamp'].quantile(q=0.8, interpolation='nearest')
time_treshold

975768738

In [6]:
train_log = log[log['timestamp'] <= time_treshold]
test_log = log[log['timestamp'] > time_treshold]
print(train_log.shape[0], test_log.shape[0])

800168 200041


In [296]:
users_union = set(test_log['user_id']) & set(train_log['user_id'])
print(test_log['user_id'].nunique(), len(users_union))

1783 1143


In [297]:
test_users = sorted(list(users_union))

More examples of splitting are available [here](https://github.com/sb-ai-lab/RePlay/blob/main/examples/04_splitters.ipynb)

## 2. Metrics

In [293]:
K = 10

#### HitRate

$$HitRate@K(i) = \max_{j \in [1..K]}\mathbb{1}_{r_{ij}}$$


$$ HitRate@K = \frac{\sum_{i=1}^{N}HitRate@K(i)}{N} $$

$\mathbb{1}_{r_{ij}}$ -- indicator function stating that user $i$ interacted with item $j$

In [143]:
def user_hr(row):
    """
    Calculate HitRate value.

    'row' contains
        a list of ground truth items in ``gt_items`` and
        a list of recommended items in ``pred_list``.
    """
    for item in row['pred_list']:
        if item in row['gt_list']:
            return 1
    return 0

#### Coverage

$$Coverage@K=\frac{\left|\bigcup\limits_{u\in U} y_u\right|}{|I|}$$


In [190]:
def coverage(pred, k, all_items=train_log['item_id']):
    pred_to_consider = set(leave_top_k(pred, k)['item_id'].values)
    all_items = set(all_items.values)
    return len(pred_to_consider & all_items) / len(all_items)

#### Wrapping

In [9]:
def metric_wrap(pred, ground_truth, k, metric_by_user):
    """
    Prepare data for metric calculation (create dataframe with columns 'user_id', 'pred_list', 'gt_list').

    'pred_list' is a list of top-k recommendation ordered by relevance (most relevant is the first)
    'gt_list' is a list of items from tests data.
    Return mean metric value and dataframe with metric value for each user
    """
    pred_cropped = leave_top_k(pred, k)
    # prepare score lists
    pred_grouped = (pred_cropped
                .sort_values(['user_id', 'rating'], ascending=[False, False])
                .groupby('user_id')['item_id']
                .apply(list).rename('pred_list')
               )
    gt_grouped = ground_truth.groupby('user_id')['item_id'].apply(list).rename('gt_list')
    to_compare = gt_grouped.to_frame().join(pred_grouped, how='left')
    to_compare['pred_list'] = to_compare['pred_list'].apply(lambda x: x if isinstance(x, list) else [])
    # compute metric
    metric_by_user = to_compare.apply(metric_by_user, axis=1)
    return metric_by_user.mean(), metric_by_user

In [10]:
def leave_top_k(pred: pd.DataFrame,
                 k: int=K,
                 group_by_col: str='user_id',
                 order_by_col: str='rating') -> pd.DataFrame:
    """
    crop predictions to leave top-k recommendations for each user
    """
    if pred.groupby(group_by_col)[group_by_col].count().max() <= k:
        return pred
    cropped_pred = deepcopy(pred)
    cropped_pred['rank'] = (cropped_pred
                            .groupby(group_by_col)[[order_by_col]]
                            .rank(method="first", ascending=False))
    cropped_pred = cropped_pred[cropped_pred['rank'] <= k].drop(columns=['rank'])
    return cropped_pred

In [ ]:
def measure(pred, true, name, df=None, cov_items=train_log['item_id']):
    if df is None:
        df = pd.DataFrame(columns=['hit_rate@K', 'coverage@K'])
    df.loc[name, 'hit_rate@K'] = metric_wrap(pred=pred, ground_truth=true, k=K, metric_by_user=user_hr)[0]

    if cov_items is not None:
        df.loc[name, 'coverage@K'] = coverage(pred=pred, k=K)
    return df

## 3. Baseline (most popular)

In [262]:
popular_items = train_log['item_id'].value_counts().head(10).index

In [278]:
users = []
items = []
ratings = []

for i, user in enumerate(test_users):
    users.extend([user] * 10)
    items.extend(popular_items)
    ratings.extend([1] * 10)

In [279]:
popular_preds = pd.DataFrame({'user_id': users, 'item_id': items, 'rating': ratings})

In [294]:
metrics = measure(popular_preds, test_log, 'PopRec')
metrics.sort_values('hit_rate@K', ascending=False)

,hit_rate@K,coverage@K
PopRec,0.286596,0.002731


## 4. EASE

$$r_{ui} = R_{u,\cdot}\cdot W_{\cdot, i}$$

$$P = \left(R^TR + \lambda E\right)^{-1}$$

\begin{equation*}
W_{ij} =
    \begin{cases}
      0, \text{if } i = j\\
      -\frac{P_{ij}}{P_{jj}}, \text{otherwise}\\
    \end{cases}\,
\end{equation*}

#### Get weight matrix

In [238]:
def compute_weight_matrix(rating_matrix, lambd=1000):
    raise NotImplementedError

In [239]:
test_data = csr_matrix(np.random.randint(low=0, high=1, size=(100, 200)))
test_weight = compute_weight_matrix(test_data)
assert test_weight.shape == (test_data.shape[1], test_data.shape[1])
assert np.diagonal(test_weight) == np.zeros(200)

In [240]:
%%time
user_num = train_log["user_id"].max() + 1
item_num = train_log["item_id"].max() + 1

train_mat = defaultdict(float)
for _, user, item, rel in train_log[["user_id", "item_id", "rating"]].itertuples():
    train_mat[user, item] = rel
rating_matrix = dok_matrix((user_num, item_num), dtype=np.float32)
dict.update(rating_matrix, train_mat)

CPU times: user 1.34 s, sys: 141 ms, total: 1.48 s
Wall time: 1.69 s


In [241]:
%%time
weight_matrix = compute_weight_matrix(rating_matrix)
print(weight_matrix.shape)
weight_matrix

(3953, 3953)
CPU times: user 10.6 s, sys: 629 ms, total: 11.2 s
Wall time: 8.24 s


matrix([[ 0.        , -0.        , -0.        , ..., -0.        ,
         -0.        , -0.        ],
        [-0.        ,  0.        ,  0.01122821, ..., -0.00011326,
         -0.00117648,  0.00189516],
        [-0.        ,  0.03482976,  0.        , ..., -0.00119722,
          0.00373784, -0.00438449],
        ...,
        [-0.        , -0.00092179, -0.00314107, ...,  0.        ,
          0.01322464,  0.0241104 ],
        [-0.        , -0.00873231,  0.00894391, ...,  0.01206109,
          0.        ,  0.05614798],
        [-0.        ,  0.00671349, -0.00500704, ...,  0.01049453,
          0.02679723,  0.        ]])

In [242]:
assert weight_matrix.shape == (train_log["item_id"].max() + 1, train_log["item_id"].max() + 1)

#### Score items

In [243]:
%%time
scores = rating_matrix.dot(weight_matrix)
scores

CPU times: user 7.78 s, sys: 222 ms, total: 8.01 s
Wall time: 10 s


matrix([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.        ,  0.65172042, -0.23183205, ...,  0.01003256,
         -0.07928256,  0.11351429],
        [ 0.        ,  0.62518294,  0.1274956 , ...,  0.10665428,
          0.00890156,  0.09359003],
        [ 0.        ,  1.15330135,  0.13435063, ...,  0.02260813,
         -0.14015023,  0.08526628]])

Filter seen items

In [244]:
scores = scores - rating_matrix * 1e6
scores

matrix([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        ...,
        [ 0.00000000e+00,  6.51720423e-01, -2.31832046e-01, ...,
          1.00325612e-02, -7.92825618e-02,  1.13514295e-01],
        [ 0.00000000e+00,  6.25182939e-01,  1.27495604e-01, ...,
          1.06654281e-01,  8.90155535e-03,  9.35900256e-02],
        [ 0.00000000e+00, -2.99999885e+06,  1.34350631e-01, ...,
          2.26081279e-02, -1.40150232e-01,  8.52662766e-02]])

#### Prediction

In [272]:
test_scores = scores[test_users]
test_scores

matrix([[ 0.00000000e+00, -2.17254915e-01,  1.17194376e-01, ...,
          4.59303995e-02,  2.01657586e-01,  7.56078503e-01],
        [ 0.00000000e+00,  7.89859697e-01,  1.88756837e-01, ...,
          5.23780404e-02, -5.72012530e-03, -2.76379148e-02],
        [ 0.00000000e+00,  2.98297292e-01, -3.99999883e+06, ...,
         -1.33029174e-02,  1.45984859e-02, -5.40161435e-03],
        ...,
        [ 0.00000000e+00, -3.99999655e+06,  8.67747401e-01, ...,
         -6.03502812e-03,  3.09297924e-02,  9.69739180e-02],
        [ 0.00000000e+00,  3.21665201e-01, -4.24920208e-02, ...,
          5.63295854e-03, -2.02729367e-02,  1.60623935e-01],
        [ 0.00000000e+00, -2.99999885e+06,  1.34350631e-01, ...,
          2.26081279e-02, -1.40150232e-01,  8.52662766e-02]])

In [273]:
top_k_preds = np.argsort(-test_scores)[:,:10].tolist()
top_k_preds[-2:]

[[589, 457, 1196, 1198, 2947, 377, 3527, 110, 1222, 2194],
 [1270, 1267, 2064, 1233, 1186, 1221, 1387, 1211, 1207, 3006]]

In [274]:
top_k_scores = -np.sort(-test_scores)[:,:10]
top_k_scores = top_k_scores.tolist()
top_k_scores[-2:]

[[2.603474939018941,
  2.489681291802156,
  2.2029600392412387,
  1.988692232579327,
  1.9653669549020023,
  1.919907170657844,
  1.9063918080762208,
  1.8163945633831358,
  1.3705816064097764,
  1.3629720180855718],
 [2.5271285814109268,
  2.460725476933279,
  2.353463953472486,
  2.167720765912828,
  2.159128010689464,
  1.9781180210653846,
  1.8466103545949273,
  1.7845591273484922,
  1.7773776885654777,
  1.7255389533329972]]

In [275]:
users = []
items = []
ratings = []

for i, user in enumerate(test_users):
    users.extend([user] * 10)
    items.extend(top_k_preds[i])
    ratings.extend(top_k_scores[i])

In [276]:
ease_preds = pd.DataFrame({'user_id': users, 'item_id': items, 'rating': ratings})
ease_preds

,user_id,item_id,rating
0,635,3897,1.445817
1,635,923,1.211050
2,635,3504,1.172587
3,635,1221,1.158296
4,635,1193,1.089239
...,...,...,...
11425,6040,1221,1.978118
11426,6040,1387,1.846610
11427,6040,1211,1.784559
11428,6040,1207,1.777378


## Results

In [291]:
metrics = measure(ease_preds, test_log, 'EASE', metrics)
metrics.sort_values('hit_rate@K', ascending=False)

,hit_rate@K,coverage@K
EASE,0.451486,0.255871
PopRec,0.286596,0.002731
